Decrease "chunk_size" variable in duplex_caller before running tests to ensure the flushing system works properly

In [ ]:
!python ../../dup_informed_caller.py -h

# Test on curated test case

In [ ]:
!python ../../dup_informed_caller.py 'dup_informed_caller_test_case.bam' test.out

In [ ]:
expected = open('dup_informed_caller_expected_output.txt', 'r')
actual = open('test.out', 'r')
for exp, act in zip(expected, actual):
    if exp != act:
        print('ERROR')
        print(exp)
        print(act)
    else:
        print('MATCH')
expected.close()
actual.close()

In [ ]:
with open('test.out') as f:
    for l in f:
        print(l)

# Test that --duplicate_support works

In [ ]:
!python ../../dup_informed_caller.py --chunk_size 1 --duplicate_support 4 'dup_informed_caller_test_case.bam' test.out

In [ ]:
expected = open('dup_informed_caller_dup_support_4_expected_output.txt', 'r')
actual = open('test.out', 'r')
for exp, act in zip(expected, actual):
    if exp != act:
        print('ERROR')
        print(exp)
        print(act)
    else:
        print('MATCH')
expected.close()
actual.close()

In [ ]:
with open('test.out') as f:
    for l in f:
        print(l)

# Test that results match naive_caller when run on a deduplicated sample

In [ ]:
from tqdm import tqdm

In [ ]:
# SAMtools markdup doesn't actually mark all duplicates, use this to remove all
fin = open('../../../data/align/dup_informed_test_long_only.sam', 'r')
fout = open('../../../data/align/dup_informed_test_dedup_for_real.sam', 'w')
frags = set()
for l in tqdm(fin):
    if l[0] == '@':
        fout.write(l)
        continue
    
    s = l.split('\t')
    frag = (s[2], s[3], s[8])
    if frag not in frags:
        frags.add(frag)
        fout.write(l)
        
fin.close()
fout.close()

In [ ]:
f_inf = open('../../../data/variant/dup_informed_test_informed.tsv', 'r')
f_nai = open('../../../data/variant/dup_informed_test_naive.tsv', 'r')
# TODO left off here
for inf, nai in tqdm(zip(f_inf, f_nai)):
    inf = inf.split('\t')[:10]
    inf[8] = sorted(inf[8]) # sort for comparison to naive output, since I don't care if the order is different
    inf[9] = sorted(inf[9][:-1]) # need to ignore the trailing newline
    nai = nai.split('\t')[:10]
    nai[8] = sorted(nai[8])
    nai[9] = sorted(nai[9])
    if inf != nai:
        print('ERROR')
        print(inf)
        print(nai)
        # break